# Advanced python CoP

## Table of content:
1. Tips
2. String formatting
3. `*args, **kwargs`

## Tips
---


### How to choose pip package

1. Popularity indexes:
 - https://hugovk.github.io/top-pypi-packages/
 - https://pythonwheels.com/
2. Check if project is alive on github

### When docs isn’t enough (or there isn’t any) and there’s no stackoverflow question :(

Example function usages from open source projects:
https://www.programcreek.com/python/


https://realpython.com

### Duck typing

In [29]:
class A:
    def f(self):
        return "A"


class B:
    def f(self):
        return "B"


a, b = A(), B()

In [30]:
a.__class__ = b.__class__

In [33]:
a.f()

'B'

### Monkey patching

In [36]:
a.f = lambda: "A"
a.f()

'A'

### Python is a really dynamic language, almost everything can change in runtime
len has to be evaluated each iteration, because len function could be replaced in different thread

In [199]:
from threading import Timer


def break_len():
    len = None


Timer(0.02, break_len).start()
for _ in range(10000):
    len("abc")

TypeError: 'NoneType' object is not callable

### Decorators
Simplest decorator:

In [128]:
def decorator(f):
    pass

@decorator
def decorated():
    pass

It works because @ is just a syntactic sugar for:

In [132]:
decorated = decorator(decorated)

Unfortunatelly:

In [165]:
decorated is None

True

Simplest functional decorator

In [188]:
from datetime import datetime

def call_and_print_time(func):
    now = datetime.now()
    func()
    print("Execution time: ", datetime.now() - now)

def execution_time_printed(func):
    return lambda: call_and_print_time(func)

@execution_time_printed
def f():
    print("Doing work")
    from time import sleep
    sleep(0.3)
    
f()

Doing work
Execution time:  0:00:00.300432


But since functions can be defined inside other functions it's a common practice to do it this way:

In [189]:
from datetime import datetime


def execution_time_printed(func):
    def call_and_print_time():
        now = datetime.now()
        func()
        print("Execution time: ", datetime.now() - now)
    return call_and_print_time

@execution_time_printed
def f():
    print("Doing work")
    from time import sleep
    sleep(0.3)
    
f()

Doing work
Execution time:  0:00:00.300651


Let's add arguments delegation and return value

In [190]:
from datetime import datetime


def execution_time_printed(func):
    def call_and_print_time(*args, **kwargs):
        now = datetime.now()
        return_value = func(*args, **kwargs)
        print("Execution time: ", datetime.now() - now)
        return return_value
    return call_and_print_time

@execution_time_printed
def add(x, y):
    return x + y
    
add(1, 2)

Execution time:  0:00:00.000004


3

In [191]:
add.__name__

'call_and_print_time'

In [192]:
"\U0001f622"

'😢'

We could fix this by adding `call_and_print_time.__name__ = func__name__` but there's more metadata to fix (i.e. docstring). @wraps to the rescue

In [202]:
from datetime import datetime
from functools import wraps

def execution_time_printed(func):
    @wraps(func)
    def call_and_print_time(*args, **kwargs):
        now = datetime.now()
        return_value = func(*args, **kwargs)
        print("Execution time: ", datetime.now() - now)
        return return_value
    return call_and_print_time

@execution_time_printed
def add(x, y):
    return x + y


add.__name__

'add'

Here comes the real inception - decorator with arguments have to return a decorator without args  
@**execution_time_printed(file=sys.stderr)** <- bolded must return a decorator

In [216]:
import io
import sys
from datetime import datetime
from functools import wraps


def execution_time_printed(file=sys.stdout):
    def decorator(func):
        @wraps(func)
        def call_and_print_time(*args, **kwargs):
            now = datetime.now()
            return_value = func(*args, **kwargs)
            print("Execution time: ", datetime.now() - now, file=file)
            return return_value

        return call_and_print_time

    return decorator


string_stream = io.StringIO()


@execution_time_printed(file=string_stream)
def add(x, y):
    return x + y


add(1, 2)
string_stream.seek(0)
print(f"{string_stream.read()=}")

string_stream.read()='Execution time:  0:00:00.000006\n'


But now you always needs to call the decorator

In [217]:
@execution_time_printed
def add(x, y):
    return x + y

add(1,2)

TypeError: decorator() takes 1 positional argument but 2 were given

In [218]:
@execution_time_printed()
def add(x, y):
    return x + y

add(1,2)

Execution time:  0:00:00.000005


3

In [220]:
import io
import sys
from datetime import datetime
from functools import wraps


def execution_time_printed(func_=None, file=sys.stdout):
    def decorator(func):
        @wraps(func)
        def call_and_print_time(*args, **kwargs):
            now = datetime.now()
            return_value = func(*args, **kwargs)
            print("Execution time: ", datetime.now() - now, file=file)
            return return_value

        return call_and_print_time
    if func_ is None:
        return decorator
    else:
        return decorator(func_)


@execution_time_printed
def add(x, y):
    return x + y


add(1, 2)


Execution time:  0:00:00.000005


3

Classes can also be decorated:

In [225]:
import functools

def singleton(cls):
    """Make a class a Singleton class (only one instance)"""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if not wrapper_singleton.instance:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton

@singleton
class TheOne:
    pass

a, b = TheOne(), TheOne()
a is b

True

# TODO: add class decorator

# TODO decorator with arguments and without at the same time

### Everything is an object in Python

Functions are objects

In [62]:
def f():
    if not hasattr(f, "call_count"):
        f.call_count = 0
    f.call_count += 1


f()
f()
print(f.call_count)

2


In [194]:
def Person(name):
    def new_person():
        pass
    new_person.say_hello = lambda: f"My names is {new_person.name}"
    new_person.name = name
    return new_person

In [195]:
janusz = Person(name="Janusz")

In [196]:
janusz.say_hello()

'My names is Janusz'

In [197]:
janusz.name

'Janusz'

In [114]:
def factorial(n: int)-> int:
    if n == 1:
        return 1
    else:
        return n * factorial(n-1)

In [116]:
%%timeit
factorial(1000)
factorial(900)
factorial(800)

849 µs ± 38.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [112]:
def cache_factorial(n: int) -> int:
    if not hasattr(cache_factorial, "__cache__"):
        cache_factorial.__cache__ = {1: 1}

    if n not in cache_factorial.__cache__:
        cache_factorial.__cache__[n] = n * cache_factorial(n - 1)

    return cache_factorial.__cache__[n]


In [222]:
%%timeit
cache_factorial(1000)
cache_factorial(900)
cache_factorial(800)

771 ns ± 43.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [224]:
import functools

def cache(func):
    """Keep a cache of previous function calls"""
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = dict()
    return wrapper_cache

@cache
def factorial(n):
    if n < 2:
        return n
    return factorial(n - 1) + factorial(n - 2)

# TODO: add this as decorator = lru_cache

### Typing

### TODO: dynamic class creation

TODO: black and isort

TODO: microbenchmarking

### String formatting

In [7]:
x = 10

In [2]:
"x = %d" % x  # python2 style

'x = 10'

In [3]:
"x = " + str(x)  # get's complex with multiple concatenation

'x = 10'

In [4]:
"x = {}".format(x)

'x = 10'

In [5]:
"x = {x}".format(x=x)

'x = 10'

In [6]:
f"x = {x}"  # python 3.6

'x = 10'

In [8]:
f"{x + 2 * 2}" # any valid expression works

'14'

In [8]:
"x = {x}".format(**vars())

'x = 10'

In [9]:
f"{x=}"  # python 3.8

'x=10'

In [9]:
f"{x + 2 * 2 =}" # python 3.8

'x + 2 * 2 =14'

### `*args, **kwargs`

http://python.astrotech.io/functions/kwargs-define.html

### super() TODO

In [11]:
class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)

class Cube(Square):
    def surface_area(self):
        face_area = super().area()
        return face_area * 6

    def volume(self):
        face_area = super().area()
        return face_area * self.length

NameError: name 'Rectangle' is not defined

### Multiple inheritance, method resolution order (mro)

In [ ]:
class A:
    def f(self):
        print("A")

class B:
    def f(self):
        print("B")

class C(B,A):
    pass

C().f()                             
print(C.mro())

In [ ]:
class C(A, B):
    pass

C().f()                             
print(C.mro())

In [ ]:
class C(A, B):
    def f(self):
        B.f(self) # in general class.method(self) == object.method()

C().f()

TODO: virtualenv, python -m

### Enumerate

In [ ]:
values = ["a","b"]

In [ ]:
# DONT DO THIS:
for i in range(len(values)):
    print(f"values[{i}] = {values[i]}")

In [ ]:
for index, value in enumerate(values):
    print(f"values[{index}] = {value}")

In [ ]:
list(enumerate(values)) == [(0, "a"), (1, "b")]

TODO: add map vs list comprehensions

### Iter protocol

#### For loop under the hood

In [ ]:
%%HTML
<iframe src='https://gfycat.com/ifr/YearlyWelcomeBlowfish' frameborder='0' scrolling='no' allowfullscreen width='640' height='1185'></iframe>

In [ ]:
for value in [1, 2, 3]:
    print(value)

In [ ]:
it = iter([1, 2, 3])
try:
    while True:
        value = next(it)
        print(value)
except StopIteration:
    pass

How to create iterable object? Implement `__iter__` that returns object implementing `__next__`

In [ ]:
class Counter:
    def __init__(self, low, high):
        self.current = low
        self.high = high

    def __iter__(self):
        return self

    def __next__(self):
        if self.current > self.high:
            raise StopIteration
        else:
            self.current += 1
            return self.current - 1


for i in Counter(3, 8):
    print(i)

You don't have to return self in `__iter__`

In [ ]:
class Counter:
    def __iter__(self):
        return iter([1, 2, 3])

for value in Counter():
    print(value)

### Generators

Generator is just like a container, but values are generated on the fly as you iterate

In [16]:
generator = range(10000000)
big_list = list(generator)

from sys import getsizeof

print(getsizeof(generator))
print(getsizeof(big_list))

48
80000056


Generator comprehensions:

In [32]:
%%timeit
power_2 = [i**2 for i in range(10**6)]

301 ms ± 7.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit
power_2_gen = (i**2 for i in range(10**6))

543 ns ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Yield

In [34]:
def primitive_range(start: int, stop: int, step: int = 1):
    current = start
    while current < stop:
        yield current
        current += step


for i in primitive_range(0, 4):
    print(i)

0
1
2
3


In [41]:
def infinite_power_2_gen():
    current = 2
    while True:
        yield current
        current *= 2

powers_of_2 = infinite_power_2_gen()
# wont' work:
# first_4 = powers_of_2[:4]
from itertools import islice
first_4_powers_gen = islice(powers_of_2, 4)
list(first_4_powers_gen)

[2, 4, 8, 16]

### Itertools

In [1]:
import itertools

In [51]:
list(itertools.chain([1, 2], (3, 4, 5), "6"))

[1, 2, 3, 4, 5, '6']

In [53]:
list(itertools.repeat(1,5))

[1, 1, 1, 1, 1]

In [3]:
%load_ext blackcellmagic

In [5]:
list(itertools.islice(itertools.count(), 4))

[0, 1, 2, 3]

"My function is not called" :(

In [40]:
def useful_function():
    print("""Nie ma czegoś takiego jak publiczne pieniądze. 
          Jeśli rząd mówi, że komuś coś da, to znaczy, że zabierze tobie, 
          bo rząd nie ma żadnych własnych pieniędzy""")
    yield 1
    # lots of other code
    
useful_function()

<generator object useful_function at 0x7f3d12da5b30>

### Context managers - RAII in Python

In [ ]:
with open("irrelevant.txt","w") as file:
    file.write("raii")

Is better than:

In [12]:
file = None
try:
    file = open("irrelevant.txt","w")
    file.write("raii")
finally:
    if file:
        file.close()

#### How does this work under the hood?

In [13]:
class File:
    def __init__(self, name: str, mode: str = "r"):
        self.name = name
        self.mode = mode
        self.file_handle = None

    def __enter__(self):
        self.file_handle = open(self.name, self.mode)
        return self.file_handle

    def __exit__(self, exc_type, exc_val, exc_tb):
        print("__exit__ called")
        if self.file_handle:
            self.file_handle.close()


with File("irrelevant.txt", "r") as f:
    10 / 0

__exit__ called


ZeroDivisionError: division by zero

### contextlib

In [ ]:
from contextlib import contextmanager


@contextmanager
def File(name: str, mode: str = "r"):
    file_handle = None
    try:
        file_handle = open(name, mode)
        yield file_handle
    finally:
        if file_handle:
            file_handle.close()

In [ ]:
with File("irrelevant.txt", "r") as f:
    10 / 0

In [ ]:
with File("3.txt", "r") as f:
    10 / 0

In [ ]:
import sys
import datetime
from typing import Generator
from typing.io import TextIO
from contextlib import contextmanager



@contextmanager
def execution_time_printed(file: TextIO = sys.stdout) -> Generator[None, None, None]:
    start = datetime.datetime.now()
    yield
    print("Execution time:", datetime.datetime.now() - start, file=file)

In [ ]:
with execution_time_printed():
    print("inside")
    import time
    time.sleep(0.5)
print("outside")

### Reentrant contextmanagers

In [ ]:
file = open("irrelevant.txt","w")
with file:
    file.write("a")

    
with file:
    file.write("a")